In [ ]:
!pip install transformers torch accelerate

In [2]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [3]:
from datasets import load_dataset

In [4]:
dataset = load_dataset("rotten_tomatoes")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [5]:
# Model yükleme
from transformers import AutoModelForSequenceClassification

In [6]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# Veri Ön İşleme
from transformers import AutoTokenizer

In [8]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [9]:
def tokenizer_dataset(dataset):
  return tokenizer(dataset["text"])

In [10]:
dataset = dataset.map(tokenizer_dataset, batched=True)

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [11]:
from transformers import DataCollatorWithPadding

In [12]:
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

In [13]:
# Hiperparametreleri Ayarlama
from transformers import TrainingArguments


In [ ]:
pip install transformers[torch]

In [15]:
training_args = TrainingArguments(
    output_dir = "my_bert_model",  # Modelin eğitildikten sonra kaydedileceği dizin
    learning_rate = 2e-5,            # Öğrenme oranı, modelin güncellenme hızını belirler
    per_device_train_batch_size = 8,  # Eğitim sırasında her cihazda kullanılacak örnek sayısı
    per_device_eval_batch_size = 8,   # Değerlendirme sırasında her cihazda kullanılacak örnek sayısı
    num_train_epochs = 2,            # Eğitim için toplam dönem sayısı
    report_to = "none"               # Eğitim sırasında raporlama yapılıp yapılmayacağını belirler
)


In [16]:
# Model Eğitimi
from transformers import Trainer

In [18]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = dataset["train"],
    eval_dataset = dataset["test"],
    tokenizer = tokenizer,
    data_collator = data_collator
)

In [19]:
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.462600
1000,0.398200
1500,0.266600
2000,0.260600


TrainOutput(global_step=2134, training_loss=0.34203696496707875, metrics={'train_runtime': 145.4059, 'train_samples_per_second': 117.327, 'train_steps_per_second': 14.676, 'total_flos': 195962231026632.0, 'train_loss': 0.34203696496707875, 'epoch': 2.0})

In [20]:
text = "I love NLP. It's fun to analyze the NLP tasks with HuggingFace"

In [21]:
inputs = tokenizer(text, return_tensors = "pt")
inputs

{'input_ids': tensor([[  101,  1045,  2293, 17953,  2361,  1012,  2009,  1005,  1055,  4569,
          2000, 17908,  1996, 17953,  2361,  8518,  2007, 17662, 12172,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [22]:
model_path = "/content/my_bert_model/checkpoint-2000"

In [24]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_path, num_labels = 2
)

In [25]:
import torch

In [26]:
with torch.no_grad():
  logits = model(**inputs).logits

In [27]:
logits.argmax().item() # model textin duygusunu tahmin etti

1